In [ ]:
! rocm-smi --showproductname

In [ ]:
! apt show rocm-core -a 

In [ ]:
import torch
print(f"number of GPUs: {torch.cuda.device_count()}")
print([torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())])

In [4]:
! pip install --upgrade pip
! pip install --upgrade pandas

In [5]:
! pip install --upgrade scikit-learn

In [6]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
num_epochs = 10
lr = 3e-4
batch_size = 128
hidden_size = 32
embd_dim = 16

In [8]:
columns = ["label", *(f"I{i}" for i in range(1, 14)), *(f"C{i}" for i in range(1, 27))]
df = pd.read_csv(
    "../data/dac_sample.txt", sep="\t", names=columns
).fillna(0)

In [9]:
# Preprocessing
sparse_cols = ["C" + str(i) for i in range(1, 27)]
dense_cols = ["I" + str(i) for i in range(1, 14)]

In [10]:
data = df[sparse_cols + dense_cols]
data = data.astype(str)
for feat in sparse_cols:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_cols] = mms.fit_transform(data[dense_cols])

In [11]:
print(data.sample(5))

        C1   C2     C3     C4   C5  C6    C7  C8  C9   C10  ...        I4  \
17040    8  130   7703   6127   23  10  4116  48   2  7709  ...  0.014388   
76247  300   17  17693   9570   23   5  3896  86   2  2505  ...  0.023981   
77532  195  428  20608  20653   23   0  1923  86   2    62  ...  0.000000   
8959   283  126      0      0  103   5  2124   8   2  3178  ...  0.000000   
18872  250   21  29261  13191   23   0  3686   8   0  2505  ...  0.004796   

             I5        I6        I7        I8        I9       I10       I11  \
17040  0.000009  0.000368  0.000114  0.001283  0.000474  0.166667  0.009615   
76247  0.002769  0.014487  0.000227  0.004704  0.007898  0.000000  0.009615   
77532  0.000136  0.000000  0.000000  0.000214  0.000079  0.000000  0.000000   
8959   0.017018  0.004911  0.000227  0.000000  0.003791  0.000000  0.009615   
18872  0.018101  0.000000  0.000000  0.002780  0.013506  0.000000  0.000000   

       I12       I13  
17040  0.0  0.000915  
76247  0.0  0.00

In [12]:
# Get the number of categories for each categorical feature
num_categories = [len(data[c].unique()) for c in sparse_cols]

# Only keep categorical features with less than 10K categories
indices_to_keep = [i for i, num in enumerate(num_categories) if num <= 10000]
num_categories_kept = [num_categories[i] for i in indices_to_keep]
sparse_cols_kept = [sparse_cols[i] for i in indices_to_keep]

In [13]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    data, df["label"], test_size=0.2, random_state=42
)

In [14]:
# Convert to tensor
# train
X_train_sparse = torch.tensor(X_train[sparse_cols_kept].values, dtype=torch.long).to(
    device
)
X_train_dense = torch.tensor(X_train[dense_cols].values, dtype=torch.float).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.float).unsqueeze(1).to(device)

# test
X_test_sparse = torch.tensor(X_test[sparse_cols_kept].values, dtype=torch.long).to(device)
X_test_dense = torch.tensor(X_test[dense_cols].values, dtype=torch.float).to(device)
y_test = torch.tensor(y_test.values, dtype=torch.float).unsqueeze(1).to(device)


In [15]:
# Create DataLoader
train_dataset = TensorDataset(X_train_sparse, X_train_dense, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# Create DataLoader for test data
test_dataset = TensorDataset(X_test_sparse, X_test_dense, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [16]:
class FeatureInteraction(nn.Module):
    def __init__(self):
        super(FeatureInteraction, self).__init__()

    def forward(self, x):
        feature_dim = x.shape[1]

        concat_features = x.view(-1, feature_dim, 1)
        dot_products = torch.matmul(concat_features, concat_features.transpose(1, 2))
        ones = torch.ones_like(dot_products)

        mask = torch.triu(ones)
        out_dim = feature_dim * (feature_dim + 1) // 2

        flat_result = dot_products[mask.bool()]
        reshape_result = flat_result.view(-1, out_dim)

        return reshape_result


In [17]:
class DLRM(torch.nn.Module):

    def __init__(
        self,
        embd_dim,
        num_categories,
        num_dense_feature,
        hidden_size,
    ):
        super(DLRM, self).__init__()
        # create embedding for each categorical feature with the same embedding dimension
        self.embeddings = nn.ModuleList(
            [nn.Embedding(num_cat, embd_dim) for num_cat in num_categories]
        )

        self.feat_interaction = FeatureInteraction()
        self.bottom_mlp = nn.Sequential(
            nn.Linear(in_features=num_dense_feature, out_features=hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, embd_dim),
        )
        num_feat = (
            len(num_categories) * embd_dim + embd_dim
        )  # categorical and dense features
        num_feat_interact = num_feat * (num_feat + 1) // 2  # interaction features
        top_mlp_in = (
            num_feat_interact + embd_dim
        )  # interaction concat with dense features
        self.top_mlp = nn.Sequential(
            nn.Linear(in_features=top_mlp_in, out_features=hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
        )

    def forward(self, x_cat, x_num):
        B = x_cat.shape[0]
        num_sparse_feat = x_cat.shape[1]

        # look up embedding for categorical features
        embed_x = torch.concat(
            [
                self.embeddings[i](x_cat[:, i]).unsqueeze(1)
                for i in range(num_sparse_feat)
            ]
        )  # B, num_sparse_feat, embedding dim
        embed_x = embed_x.view(B, -1)  # B, num_sparse_feat * embedding dim

        # get bottom dense features
        dense_x = self.bottom_mlp(x_num)  # B, embedding dim
        # concatenate with embeddings
        x = torch.concat(
            [embed_x, dense_x], dim=-1
        )  # B, (num_sparse_feat+1) * embedding dim
        # get 2nd order interaction features
        x = self.feat_interaction(x)  # B, n*(n+1) // 2
        # combine with dense features
        x = torch.concat([x, dense_x], dim=-1)
        # pass through top mlp
        x = self.top_mlp(x)  # B, 1
        return x


In [18]:
# Define the model, loss function and optimizer
model = DLRM(
    embd_dim=embd_dim,
    num_categories=num_categories_kept,
    num_dense_feature=len(dense_cols),
    hidden_size=hidden_size,
)
model.to(device)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

/opt/conda/envs/py_3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
print(f"running on {device}")
print(sum(p.numel() for p in model.parameters()) / 1e6, "M parameters")
print(model)


running on cuda
2.195553 M parameters
DLRM(
  (embeddings): ModuleList(
    (0): Embedding(541, 16)
    (1): Embedding(497, 16)
    (2): Embedding(145, 16)
    (3): Embedding(12, 16)
    (4): Embedding(7623, 16)
    (5): Embedding(257, 16)
    (6): Embedding(3, 16)
    (7): Embedding(3799, 16)
    (8): Embedding(2796, 16)
    (9): Embedding(26, 16)
    (10): Embedding(5238, 16)
    (11): Embedding(10, 16)
    (12): Embedding(2548, 16)
    (13): Embedding(1303, 16)
    (14): Embedding(4, 16)
    (15): Embedding(11, 16)
    (16): Embedding(14, 16)
    (17): Embedding(51, 16)
    (18): Embedding(9527, 16)
  )
  (feat_interaction): FeatureInteraction()
  (bottom_mlp): Sequential(
    (0): Linear(in_features=13, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
  )
  (top_mlp): Sequential(
    (0): Linear(in_features=51376, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=1, bias=True)
  )
)


Training

In [20]:
def train_one_epoch():
    model.train()
    for i, (x_sparse, x_dense, y) in enumerate(tqdm(train_loader)):
        x_sparse = x_sparse.to(device)
        x_dense = x_dense.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        logits = model(x_sparse, x_dense)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()


In [21]:
def evaluate(dataloader, dataname):
    model.eval()
    total_samples = 0
    total_loss = 0
    total_correct = 0
    with torch.no_grad():
        for i, (x_sparse, x_dense, y) in enumerate(tqdm(dataloader)):
            x_sparse = x_sparse.to(device)
            x_dense = x_dense.to(device)
            y = y.to(device)
            logits = model(x_sparse, x_dense)
            probs = torch.sigmoid(logits)
            predictions = (probs > 0.5).long()

            loss = criterion(logits, y)
            total_loss += loss.item() * y.shape[0]
            total_correct += (predictions == y).sum().item()
            total_samples += y.shape[0]

    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples * 100
    print(
        f"{dataname} accuracy = {accuracy:0.2f}%, {dataname} avg loss = {avg_loss:.6f}"
    )
    return accuracy, avg_loss

In [22]:
for epoch in range(num_epochs):
    print(f"epoch {epoch+1}")
    train_one_epoch()
    evaluate(train_loader, "train")
    evaluate(test_loader, "test")
    print()

epoch 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [00:01<00:00, 326.16it/s]


train accuracy = 77.33%, train avg loss = 0.527628


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 329.75it/s]


test accuracy = 77.05%, test avg loss = 0.531474

epoch 2


...

test accuracy = 77.14%, test avg loss = 0.501504

epoch 10


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [00:01<00:00, 322.96it/s]


train accuracy = 77.42%, train avg loss = 0.497777


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 326.68it/s]

test accuracy = 77.06%, test avg loss = 0.501636



Inference

In [23]:
# Assume we use the first 10 rows of the dataset as ad candidates
num_ads = 10
df_c = pd.DataFrame(data.iloc[0:num_ads])
# get the ad candidate features
df_ads = df_c[df_c.columns[26:39]]
# get the user features of the first row
df_user = df_c[df_c.columns[0:26]].iloc[0:1]
# replicate the user feature across all ad candidate rows
df_user_rep = df_user
for i in range(num_ads-1): 
    df_user_rep = pd.concat([df_user_rep, df_user], ignore_index=True, sort=False)
df_candidates = pd.concat([df_user_rep, df_ads], axis=1)

# Convert the feature vectors to tensor
X_inf_sparse = torch.tensor(df_candidates[sparse_cols_kept].values, dtype=torch.long).to(device)
X_inf_dense = torch.tensor(df_candidates[dense_cols].values, dtype=torch.float).to(device)

# create data loader for inferencing
y_dummy = torch.tensor([0]*num_ads, dtype=torch.float).unsqueeze(1).to(device)
inf_dataset = TensorDataset(X_inf_sparse, X_inf_dense, y_dummy)
inf_loader = DataLoader(inf_dataset, batch_size=num_ads, shuffle=True)




In [24]:
def recommend():
    with torch.no_grad():
        for i, (x_sparse, x_dense, y) in enumerate(tqdm(inf_loader)):
            x_sparse = x_sparse.to(device)
            x_dense = x_dense.to(device)
            logits = model(x_sparse, x_dense)
            probs = torch.sigmoid(logits)
    print(probs)
    return torch.max(probs, dim=0).indices[0].item()

In [25]:
print('Best ad candidate is ad', recommend())

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 315.29it/s]

tensor([[0.1380],
        [0.2414],
        [0.3493],
        [0.3500],
        [0.1807],
        [0.3009],
        [0.2203],
        [0.3639],
        [0.1890],
        [0.3702]], device='cuda:0')
Best ad candidate is ad 9
